<a href="https://colab.research.google.com/github/unique-subedi/gene-expression/blob/main/Unique_gene_expression_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import os
import datetime
import time
import math
import numpy as np
from numpy import linalg as LA
import pandas as pd
import urllib.request

In [100]:
!pip install pyreadr
import pyreadr
urllib.request.urlretrieve("https://raw.githubusercontent.com/unique-subedi/gene-expression/main/data/brain.rda", "brain.rda")
brain = pyreadr.read_r("brain.rda")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [101]:
expression = pd.DataFrame(brain["expression"])
genes = pd.DataFrame(brain["genes"])
samples = pd.DataFrame(brain["samples"])

In [ ]:
expression.head()
genes.head()
samples.iloc[:20,:]

`michigan` dataset contains the measurements, done at University of Michigan lab, of anterior cortex and dorsolateral cortex for each patient. 

`michigan_diff` subtracts measurement of anterior cortex from the dorsolateral cortex for each patient.

In [103]:
michigan = expression[(expression.index.str[5] == 'M') & (expression.index.str[3] != 'c') ]
michigan_diff = michigan.diff().iloc[1::2,:]
michigan_diff.index = "patient" + michigan_diff.index.str[:2]
michigan_diff

,1000_at,1001_at,1002_f_at,1003_s_at,1004_at,1005_at,1006_at,1007_s_at,1008_f_at,1009_at,...,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,AFFX-YEL002c/WBP1_at,AFFX-YEL018w/_at,AFFX-YEL021w/URA3_at,AFFX-YEL024w/RIP1_at,AFFX-hum_alu_at
patient01,1.683506,1.078551,0.954802,1.156709,0.982425,1.161395,0.890784,1.828306,1.124522,2.052794,...,0.732205,0.717167,0.600567,0.713971,0.576473,0.703452,0.676053,0.591423,0.763982,1.062522
patient02,0.996391,0.544034,0.358586,0.683919,0.587913,0.984146,0.349759,0.993259,1.090844,1.364808,...,0.594352,0.432579,0.375512,0.516236,0.421621,0.533885,0.481034,0.352948,0.443977,0.724038
patient03,0.748157,0.410856,0.375251,0.664853,0.519776,0.806980,0.566739,0.900413,1.118494,1.007720,...,0.504707,0.452460,0.378896,0.451803,0.356578,0.436162,0.509169,0.479238,0.485842,0.763825
patient04,0.965669,0.526189,0.348966,0.575349,0.353175,0.445199,0.361394,1.390621,0.703409,1.171899,...,0.332539,0.311592,0.209239,0.335892,0.255916,0.270513,0.342929,0.264286,0.342026,0.986904
patient05,0.805991,0.436910,0.485836,0.179388,0.270416,0.651277,0.387753,0.223662,0.560202,0.392694,...,0.396176,0.366402,0.532929,0.333441,0.287466,0.428850,0.365462,0.462509,0.414235,-0.063161
patient06,0.523212,0.548558,0.332666,0.131101,0.105857,0.221342,0.260256,0.164966,-0.164174,0.405472,...,0.330896,0.188302,0.309652,0.365562,0.230068,0.495998,0.621407,0.600562,0.542592,-0.431859
patient07,0.744211,0.529751,0.570855,0.402839,0.382700,0.699813,0.474074,0.725456,0.385778,0.179182,...,0.616852,0.600624,0.649939,0.529744,0.603646,0.613554,0.463463,0.596304,0.379135,0.208659
patient08,0.307347,0.097159,0.261037,-0.244442,-0.034621,-0.026341,-0.008480,0.135303,-0.040331,-0.086372,...,0.055827,0.091226,0.225278,0.022298,0.051644,0.089153,-0.030807,0.369845,-0.048141,0.075750
patient09,0.632714,0.512219,0.452847,0.249875,0.268599,0.495261,0.314457,0.360264,0.214071,-0.038456,...,0.502343,0.394397,0.472787,0.441500,0.462422,0.510486,0.483740,0.581277,0.485722,0.255156
patient10,0.817703,0.410601,0.404771,0.237337,0.246678,0.382905,0.382378,0.391850,0.231995,0.225793,...,0.451583,0.405504,0.511481,0.502447,0.503460,0.519477,0.444961,0.559787,0.526603,0.080064


In [130]:
(michigan_diff.loc['patient08']< 0).sum()

4176

In [133]:
michigan_diff.loc[:, '33942_s_at']

patient01    3.089106
patient02    1.849506
patient03    1.621996
patient04    1.514725
patient05    1.296588
patient06    1.890518
patient07    1.154332
patient08    0.658243
patient09    1.048392
patient10    1.169494
Name: 33942_s_at, dtype: float64

Out of 126000 entries in this dataset, only 6764 (~5%) of the entries are negative. That means, the measurement for dorsolateral region is generally larger than that of anterior region for every patient.

In [104]:
126000 - (michigan_diff > 0).sum().sum()

6764

Take absolute values of each difference and compute the average over all patients for each probe sets and obtain top 20 probe sets with highest magnitude.

In [134]:
top_probes_michigan = michigan_diff.abs().mean().sort_values(ascending = False).iloc[:100]
top_probes_michigan

38308_g_at    1.700100
33925_at      1.581787
33942_s_at    1.529290
36608_at      1.494925
35742_at      1.435630
                ...   
41498_at      1.140076
38708_at      1.137271
32315_at      1.136706
40512_at      1.134779
35174_i_at    1.133433
Length: 100, dtype: float64

Map top probe sets to top genes using `genes` dataset. 

In [106]:
top_genes_michigan = genes.loc[top_probes_michigan.keys()].loc[:, 'sym'].values
top_genes_michigan

['NCDN', 'NRGN', 'STXBP1', 'MDH1', 'C16orf45', ..., 'HNRNPK', 'NPTN', 'FAM127A', 'OLFM1', 'VAMP2']
Length: 20
Categories (8783, object): ['AADAC', 'AAK1', 'AAMP', 'AANAT', ..., 'ZYX', 'ZZEF1', 'ZZZ3', 'psiTPTE22']

In [145]:
def get_top_genes(lab):
  df = expression[(expression.index.str[5] == lab) & (expression.index.str[3] != 'c') ]
  df_diff = df.diff().iloc[1::2,:]
  top_probes = df_diff.abs().mean().sort_values(ascending = False).iloc[:20]
  top_genes = genes.loc[top_probes.keys()].loc[:, 'sym'].values
  return np.array(top_genes)


In [148]:
top_genes_michigan = get_top_genes("M")
top_genes_irvine = get_top_genes("I")
print(np.intersect1d(top_genes_michigan, top_genes_irvine))

['NCDN' 'NRGN' 'STXBP1' 'VSNL1']
